<a href="https://colab.research.google.com/github/LaTsa99/DeepMinds/blob/master/DeepMinds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing necessary modules for data preprocessing**

In [1]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

**Downloading the zip file containing our training data**

In [2]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1UM65-NZoXAr3lydSkmX2OmHm63-PfepX -O /tmp/girls.zip

Downloading...
From: https://drive.google.com/uc?id=1UM65-NZoXAr3lydSkmX2OmHm63-PfepX
To: /tmp/girls.zip
126MB [00:00, 234MB/s]


**Unzipping the zip file containing the training data**

In [3]:
local_zip = '/tmp/girls.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

**Creating path strings to easily import the train data**

In [4]:
base_dir = os.path.join('/tmp', 'girls')

train_dir = os.path.join(base_dir, 'train')
train_bikini_dir = os.path.join(train_dir, 'bikini')
train_long_dir = os.path.join(train_dir, 'long')

valid_dir = os.path.join(base_dir, 'valid')
valid_bikini_dir = os.path.join(valid_dir, 'bikini')
valid_long_dir = os.path.join(valid_dir, 'long')

test_dir = os.path.join(base_dir, 'test')
test_bikini_dir = os.path.join(test_dir, 'bikini')
test_long_dir = os.path.join(test_dir, 'long')

**Querrying the filenames into variables**

In [5]:
bikini_train_fnames = os.listdir(train_bikini_dir)
bikini_valid_fnames = os.listdir(valid_bikini_dir)
bikini_test_fnames = os.listdir(test_bikini_dir)

long_train_fnames = os.listdir(train_long_dir)
long_valid_fnames = os.listdir(valid_long_dir)
long_test_fnames = os.listdir(test_long_dir)

**Setting the input image dimensions**

In [6]:
img_height = 299
img_width = 299

**Generating our data and making data generators**

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
test_generator  = test_datagen.flow_from_directory(test_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')

Found 1572 images belonging to 2 classes.
Found 527 images belonging to 2 classes.
Found 266 images belonging to 2 classes.
